In [1]:
from interpolant import AnyOrderMaskInsertionInterpolant
import torch
from schedule import LinearSchedule

In [63]:
mask_token = 0
pad_token = 3
max_length = 8
mask_schedule = LinearSchedule()
interpolant = AnyOrderMaskInsertionInterpolant(mask_schedule, 3, 0, 3, 8)
t = torch.Tensor([0.99])
x1 = torch.Tensor([[1, 2, 2, 2, 1, 2, 2, 3]]).to(torch.int64)
xt, st = interpolant.sample_interpolant(t, x1)


reparametrised_rate = interpolant.reparametrised_conditional_rate(xt, st, t, x1)

print("xt", xt)
print("st", st)
print("posterior", reparametrised_rate.per_token_posterior)
print("length", reparametrised_rate.length_posterior)

rate tensor([100.0001])
xt tensor([[1, 2, 2, 2, 1, 2, 2, 3]])
st tensor([[0, 1, 2, 3, 4, 5, 6, 9]])
posterior tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])
length tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.]]])


In [64]:
t = torch.Tensor([0.99])
x1 = torch.Tensor([[1, 2, 2, 2, 1, 2, 2, 3]]).to(torch.int64)
xt, st, xt_mask_indices, x1_remained, gap_counts = interpolant.jay_sample_interpolant(t, x1)

print(xt)
print(st)
print(xt_mask_indices)
print(x1_remained)
print(gap_counts)


tensor([[1, 2, 2, 2, 1, 2, 2, 3]])
tensor([[ True,  True,  True,  True,  True,  True,  True, False]])
tensor([[False, False, False, False, False, False, False, False]])
tensor([[1, 2, 2, 2, 1, 2, 2, 3]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [65]:
x1_one_hot = torch.nn.functional.one_hot(x1_remained, num_classes=4).to(xt.dtype)
# Expand to match x1_one_hot shape: (B, L, vocab_size)
mask = xt_mask_indices.unsqueeze(-1)  # (B, L, 1)
x1_one_hot = x1_one_hot * mask / (1 - t.view(-1, 1, 1))

print(x1_one_hot)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])


In [37]:
reparametrised_rate.length_posterior

tensor([[[0., 0., 0., 1., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.]]])